Relax and hold Steady part 4

Test Problem!

Poisson:

\begin{equation}
\nabla^2 p = -2\left(\frac{\pi}{2}\right)^2\sin\left( \frac{\pi x}{L} \right) \cos\left(\frac{\pi y}{L}\right)
\end{equation}

in domain:

$$\left\lbrace \begin{align*}
0 &\leq x\leq 1  \\
-0.5 &\leq y \leq 0.5 
\end{align*} \right.$$

with BCs

$$p=0 \text{ at } \left\lbrace 
\begin{align*}
x&=0\\
y&=0\\
y&=-0.5\\
y&=0.5
\end{align*} \right.$$

Assum initial state p=0

Theory

Discretized Poisson

$$\frac{p_{i+1,j}^{k}-2p_{i,j}^{k}+p_{i-1,j}^{k}}{\Delta x^2}+\frac{p_{i,j+1}^{k}-2 p_{i,j}^{k}+p_{i,j-1}^{k}}{\Delta y^2}=b_{i,j}^{k}$$

symbollically written:

\begin{equation}
A{\bf p}={\bf b}
\end{equation}

where $A$ is a $N\times N$ matrix.

Define iteration of potential:

\begin{equation}
{\bf p}^{k+1}={\bf p}^k + \alpha {\bf d}^k
\end{equation}



Assume solution exists to poisson

\begin{equation}
A \bf p = b
\end{equation}

The error at any point $i$ is the difference between the calculated value and the exact solution:

\begin{equation}
e^k_i = p^k_i - p^{exact}_i
\end{equation}

What if we recast the Poisson equation in terms of a not-perfectly-relaxed $\bf p^k$?

\begin{equation}
A \bf p^k \approx b
\end{equation}

We can write out the modified Poisson equation like this:

\begin{equation}
{\bf r^k} + A \bf p^k = b
\end{equation}

Steepest Descent:

At iteration $0$, we choose an inital guess. Unless we are immensely lucky, it will not satisfy the Poisson equation and we will have,

\begin{equation}
{\bf b}-A{\bf p}^0={\bf r}^0\ne {\bf 0}
\end{equation}

${\bf r}^0$ is called the initial residual and measures how far we are from satisfying the linear system.

Define at each iteration

\begin{equation}
{\bf r}^k={\bf b}-A{\bf p}^k
\end{equation}

In the method of steepest descent, two choices are made:

1. the direction vectors are chosen to be the residuals ${\bf d}^k = {\bf r}^k$.
2. the length of the jumps are such that the $n+1^{th}$ residual is orthogonal to the $n^{th}$ resuidual.

Condition 2 requires that,

\begin{align}
{\bf r}^{k+1}\cdot {\bf r}^{k} = 0 \nonumber \\
\Leftrightarrow ({\bf b}-A{\bf p}^{k+1}) \cdot {\bf r}^{k} = 0 \nonumber \\
\Leftrightarrow ({\bf b}-A({\bf p}^{k}+\alpha {\bf r}^k)) \cdot {\bf r}^{k} = 0 \nonumber \\
\Leftrightarrow ({\bf r}^k+\alpha A{\bf r}^k) \cdot {\bf r}^{k} = 0 \nonumber \\
\alpha = \frac{{\bf r}^k \cdot {\bf r}^k}{A{\bf r}^k \cdot {\bf r}^k}.
\end{align}



In [1]:
import numpy
from math import pi
%matplotlib inline

In [3]:
from laplace_helper import plot_3D, L2_rel_error
from cg_helper import poisson_2d, p_analytical

In [5]:
# Parameters
nx = 101
ny = 101
xmin = 0
xmax = 1
ymin = -0.5
ymax = 0.5

l2_target = 1e-10

# Spacing
dx = (xmax-xmin)/(nx-1)
dy = (ymax-ymin)/(ny-1)

# Mesh
x  = numpy.linspace(xmin,xmax,nx)
y  = numpy.linspace(ymin,ymax,ny)
X,Y = numpy.meshgrid(x,y)

# Source
L = xmax-xmin
b = -2*(pi/L)**2*numpy.sin(pi*X/L)*numpy.cos(pi*Y/L)

# Initialization
p_i  = numpy.zeros((ny,nx))

# Analytical solution
pan = p_analytical(X,Y,L)

Solution process:
    
1. Calculate the residual, $\bf r^k$, which also serves as the direction vector, $\bf d^k$
2. Calculate the step size $\alpha$
3. Update ${\bf p}^{k+1}={\bf p}^k + \alpha {\bf d}^k$



We have an equation for the residual above: 
\begin{equation}
{\bf r}^k={\bf b}-A{\bf p}^k
\end{equation}

Laplacian can be discretized:

\begin{equation}
\nabla^2 p^k = 4p^k_{i,j} - \left(p^{k}_{i,j-1} + p^k_{i,j+1} + p^{k}_{i-1,j} + p^k_{i+1,j} \right)
\end{equation}



In [6]:
def steepest_descent_2d(p, b, dx, dy, l2_target):
    r  = numpy.zeros((ny,nx)) # residual
    Ar  = numpy.zeros((ny,nx)) # to store result of matrix multiplication
    
    l2_norm = 1
    iterations = 0
    l2_conv = []
    
    # Iterations
    while l2_norm > l2_target:

        pd = p.copy()
        
        r[1:-1,1:-1] = b[1:-1,1:-1]*dx**2 + 4*pd[1:-1,1:-1] - \
            pd[1:-1,2:] - pd[1:-1,:-2] - pd[2:, 1:-1] - pd[:-2, 1:-1]
        
        Ar[1:-1,1:-1] = -4*r[1:-1,1:-1]+r[1:-1,2:]+r[1:-1,:-2]+\
            r[2:, 1:-1] + r[:-2, 1:-1]

        rho = numpy.sum(r*r)
        sigma = numpy.sum(r*Ar)
        alpha = rho/sigma

        p = pd + alpha*r
        
        # BCs are automatically enforced
        
        l2_norm = L2_rel_error(pd,p)
        iterations += 1
        l2_conv.append(l2_norm)
    
    print('Number of SD iterations: {0:d}'.format(iterations))
    return p, l2_conv

In [7]:
p, l2_conv = steepest_descent_2d(p_i.copy(), b, dx, dy, l2_target)
L2_rel_error(p, pan)

Number of SD iterations: 2


8.2250762210863876e-05

Conjugate Gradients

reduces number of jumps

\begin{equation}
\alpha = \frac{{\bf r}^k \cdot {\bf r}^k}{A{\bf d}^k \cdot {\bf d}^k}
\end{equation}

direction vectors

\begin{equation}
{\bf d}^{k+1}={\bf r}^{k+1}+\beta{\bf d}^{k}
\end{equation}

where: $\beta = \frac{{\bf r}^{k+1} \cdot {\bf r}^{k+1}}{{\bf r}^k \cdot {\bf r}^k}$.

Implementing Conjugate Gradients

update $\bf p$ according to 

\begin{equation}
{\bf p}^{k+1}={\bf p}^k + \alpha {\bf d}^k
\end{equation}

Thus, the full set of steps for the method of conjugate gradients is:

Calculate ${\bf d}^0 = {\bf r}^0$ (just the once), then

1. Calculate $\alpha = \frac{{\bf r}^k \cdot {\bf r}^k}{A{\bf d}^k \cdot {\bf d}^k}$
2. Update ${\bf p}^{k+1}$
3. Calculate ${\bf r}^{k+1} = {\bf r}^k - \alpha A {\bf d}^k$ $\ \ \ \ $(see <a href='#references'>Shewchuk (1994)</a>)
4. Calculate $\beta = \frac{{\bf r}^{k+1} \cdot {\bf r}^{k+1}}{{\bf r}^k \cdot {\bf r}^k}$
5. Calculate ${\bf d}^{k+1}={\bf r}^{k+1}+\beta{\bf d}^{k}$
6. Repeat!

In [8]:
def conjugate_gradient_2d(p, b, dx, dy, l2_target):
    r  = numpy.zeros((ny,nx)) # residual
    Ad  = numpy.zeros((ny,nx)) # to store result of matrix multiplication 
    
    l2_norm = 1
    iterations = 0
    l2_conv = []
    
    # Step-0 We compute the initial residual and 
    # the first search direction is just this residual
    
    r[1:-1,1:-1] = b[1:-1,1:-1]*dx**2 + 4*p[1:-1,1:-1] - \
        p[1:-1,2:] - p[1:-1,:-2] - p[2:, 1:-1] - p[:-2, 1:-1]
    d = r.copy()
    rho = numpy.sum(r*r)
    Ad[1:-1,1:-1] = -4*d[1:-1,1:-1]+d[1:-1,2:]+d[1:-1,:-2]+\
        d[2:, 1:-1] + d[:-2, 1:-1]
    sigma = numpy.sum(d*Ad)
    
    # Iterations
    while l2_norm > l2_target:

        pk = p.copy()
        rk = r.copy()
        dk = d.copy()
        
        alpha = rho/sigma

        p = pk + alpha*dk
        r = rk- alpha*Ad
        
        rhop1 = numpy.sum(r*r)
        beta = rhop1 / rho
        rho = rhop1
        
        d = r + beta*dk
        Ad[1:-1,1:-1] = -4*d[1:-1,1:-1] + d[1:-1,2:] + d[1:-1,:-2] + \
            d[2:, 1:-1] + d[:-2, 1:-1]
        sigma = numpy.sum(d*Ad)
        
        # BCs are automatically enforced
        
        l2_norm = L2_rel_error(pk,p)
        iterations += 1
        l2_conv.append(l2_norm)
    
    print('Number of CG iterations: {0:d}'.format(iterations))
    return p, l2_conv

In [9]:
p, l2_conv = conjugate_gradient_2d(p_i.copy(), b, dx, dy, l2_target)
L2_rel_error(p,pan)

Number of CG iterations: 2


8.2250762210862941e-05

In [10]:
p, l2_conv = poisson_2d(p_i.copy(), b, dx, dy, l2_target)

Number of Jacobi iterations: 31227


Complex Poisson Problems:

\begin{equation}
b = \sin\left(\frac{\pi x}{L}\right) \cos\left(\frac{\pi y}{L}\right) + \sin\left(\frac{6\pi x}{L}\right) \cos\left(\frac{6\pi y}{L}\right)
\end{equation}

In [11]:
b = (numpy.sin(pi*X/L)*numpy.cos(pi*Y/L)+
     numpy.sin(6*pi*X/L)*numpy.sin(6*pi*Y/L))

In [12]:
p, l2_conv = poisson_2d(p_i.copy(), b, dx, dy, l2_target)

p, l2_conv = steepest_descent_2d(p_i.copy(), b, dx, dy, l2_target)

p, l2_conv = conjugate_gradient_2d(p_i.copy(), b, dx, dy, l2_target)

Number of Jacobi iterations: 31226
Number of SD iterations: 28671
Number of CG iterations: 3
